### Загружаем данные

In [1]:
!git clone https://github.com/Arina1308/hw2_data_analysis

Cloning into 'hw2_data_analysis'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 3), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots

In [3]:
data_tr = pd.read_csv('/content/hw2_data_analysis/demand-forecasting-in-retail/train.csv')
store_loc = pd.read_csv('/content/hw2_data_analysis/demand-forecasting-in-retail/STORE_LOCATION.csv', ';')

In [4]:
display(data_tr.columns)
display(store_loc.columns)

Index(['Unnamed: 0', 'product_rk', 'store_location_rk', 'period_start_dt',
       'demand', 'PROMO1_FLAG', 'PROMO2_FLAG', 'PRICE_REGULAR',
       'PRICE_AFTER_DISC', 'NUM_CONSULTANT', 'AUTORIZATION_FLAG'],
      dtype='object')

Index(['STORE_LOCATION_RK', 'STORE_LOCATION_LVL_RK4', 'STORE_LOCATION_LVL_RK3',
       'STORE_LOCATION_LVL_RK2', 'STORE_LOCATION_LVL_RK1', 'STORE_OPEN_DTTM',
       'STORE_CLOSURE_DTTM', 'STORE_LOCATION_ADK_hashing',
       'STORE_LOCATION_ATTRIB1_hashing', 'STORE_LOCATION_ATTRIB2_hashing',
       'STORE_LOCATION_ATTRIB3_hashing', 'STORE_LOCATION_ATTRIB4_hashing',
       'STORE_LOCATION_ATTRIB5_hashing', 'STORE_LOCATION_ATTRIB6_hashing',
       'STORE_LOCATION_ATTRIB7_hashing', 'STORE_LOCATION_ATTRIB8_hashing',
       'STORE_LOCATION_ATTRIB9_hashing', 'STORE_LOCATION_ATTRIB10_hashing',
       'STORE_LOCATION_ATTRIB11_hashing', 'STORE_LOCATION_ATTRIB12_hashing',
       'STORE_LOCATION_ATTRIB13_hashing', 'STORE_LOCATION_ATTRIB14_hashing',
       'STORE_LOCATION_ATTRIB15_hashing', 'STORE_LOCATION_ATTRIB16_hashing',
       'STORE_LOCATION_ATTRIB17_hashing', 'STORE_LOCATION_ATTRIB18_hashing',
       'STORE_LOCATION_ATTRIB19_hashing', 'STORE_LOCATION_ATTRIB20_hashing',
       'STORE_LOCATION

In [5]:
store_loc.rename(columns={'STORE_LOCATION_RK': 'store_location_rk'}, inplace=True)

In [6]:
data = data_tr.merge(store_loc, on='store_location_rk')

# 1. Обработка данных

 - ### Удаляем строки и колонки



In [7]:
display(data.head())
display(data.shape)

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,STORE_LOCATION_LVL_RK4,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK1,STORE_OPEN_DTTM,STORE_CLOSURE_DTTM,STORE_LOCATION_ADK_hashing,STORE_LOCATION_ATTRIB1_hashing,STORE_LOCATION_ATTRIB2_hashing,STORE_LOCATION_ATTRIB3_hashing,STORE_LOCATION_ATTRIB4_hashing,STORE_LOCATION_ATTRIB5_hashing,STORE_LOCATION_ATTRIB6_hashing,STORE_LOCATION_ATTRIB7_hashing,STORE_LOCATION_ATTRIB8_hashing,STORE_LOCATION_ATTRIB9_hashing,STORE_LOCATION_ATTRIB10_hashing,STORE_LOCATION_ATTRIB11_hashing,STORE_LOCATION_ATTRIB12_hashing,STORE_LOCATION_ATTRIB13_hashing,STORE_LOCATION_ATTRIB14_hashing,STORE_LOCATION_ATTRIB15_hashing,STORE_LOCATION_ATTRIB16_hashing,STORE_LOCATION_ATTRIB17_hashing,STORE_LOCATION_ATTRIB18_hashing,STORE_LOCATION_ATTRIB19_hashing,STORE_LOCATION_ATTRIB20_hashing,STORE_LOCATION_ATTRIB21_hashing
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN,203,203,10,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN,203,203,10,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN,203,203,10,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN,203,203,10,1,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,B5A368A9689D233843CB7A34709BD45B,1D870936A2C35AA44CB0CF9B6210FB74,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,F2D475B39396781ED7F2D76E648AFAE4,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1

(35344, 39)

In [8]:
display(data[data.demand.isna()].store_location_rk.unique())
display(data.store_location_rk.unique())

array([ 317,  355,  380,  425,  453,  504,  517,  525,  533,  535,  540,
        555,  557,  562,  637,  644,  657,  764,  798,  862,  866,  874,
       1005, 1034, 1079, 1080, 1143, 1162, 1173, 1185, 1191, 1202, 1203,
       1281, 1316, 1326, 1328, 1347, 1363, 1380])

array([ 309,  317,  355,  380,  425,  453,  504,  517,  525,  533,  535,
        540,  555,  557,  562,  637,  644,  657,  764,  798,  862,  866,
        874, 1005, 1034, 1079, 1080, 1143, 1162, 1173, 1185, 1191, 1202,
       1203, 1281, 1316, 1326, 1328, 1347, 1363, 1380])

В тесте нет 309 магазина, поэтому выкинем все записи по нему из данных

In [9]:
data = data[data.store_location_rk != 309]

In [10]:
data.isnull().sum()[data.isnull().sum() > 0]

demand               1200
PROMO1_FLAG           170
PROMO2_FLAG           170
PRICE_REGULAR        1112
PRICE_AFTER_DISC     1117
NUM_CONSULTANT        170
AUTORIZATION_FLAG     170
dtype: int64

In [11]:
data.describe()

,Unnamed: 0,product_rk,store_location_rk,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,STORE_LOCATION_LVL_RK4,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK1
count,35329.00000,35329.000000,35329.000000,34129.000000,35159.000000,35159.0,34217.000000,34212.000000,35159.0,35159.000000,35329.000000,35329.000000,35329.0,35329.0
mean,17774.09437,49257.002576,844.467406,12.226552,0.206434,0.0,1167.679357,1155.778351,0.0,0.907677,134.203063,134.203063,10.0,1.0
std,10253.68736,19148.409051,333.117293,32.585194,0.433393,0.0,1046.828551,1057.912830,0.0,0.289486,82.732522,82.732522,0.0,0.0
min,15.00000,40369.000000,317.000000,0.000000,0.000000,0.0,49.000000,8.647059,0.0,0.000000,25.000000,25.000000,10.0,1.0
25%,8893.00000,40370.000000,535.000000,2.000000,0.000000,0.0,284.290000,199.000000,0.0,1.000000,39.000000,39.000000,10.0,1.0
50%,17778.00000,40372.000000,862.000000,6.000000,0.000000,0.0,1000.000000,1000.000000,0.0,1.000000,189.000000,189.000000,10.0,1.0
75%,26651.00000,46272.000000,1173.000000,12.000000,0.000000,0.0,2000.000000,2000.000000,0.0,1.000000,203.000000,203.000000,10.0,1.0
max,35541.00000,96212.000000,1380.000000,1160.000000,2.000000,0.0,3000.000000,3000.000000,0.0,1.000000,258.000000,258.000000,10.0,1.0


Удалим следующие колонки: 'STORE_LOCATION_LVL_RK3, STORE_LOCATION_LVL_RK2, STORE_LOCATION_LVL_RK1, PROMO2_FLAG, NUM_CONSULTANT. В данных признаках все значения одинаковые. 




In [12]:
data.set_axis(data['Unnamed: 0'], inplace=True)

In [13]:
data.drop(['STORE_LOCATION_LVL_RK2', 'STORE_LOCATION_LVL_RK1', 'STORE_LOCATION_LVL_RK3', 
           'PROMO2_FLAG', 'NUM_CONSULTANT', 'Unnamed: 0'], axis=1, inplace=True)

In [14]:
data[data.columns[data.dtypes == 'object']].describe()
# очень много категориальных признаков с одним уникальным значение => удалим их 

,period_start_dt,STORE_OPEN_DTTM,STORE_CLOSURE_DTTM,STORE_LOCATION_ADK_hashing,STORE_LOCATION_ATTRIB1_hashing,STORE_LOCATION_ATTRIB2_hashing,STORE_LOCATION_ATTRIB3_hashing,STORE_LOCATION_ATTRIB4_hashing,STORE_LOCATION_ATTRIB5_hashing,STORE_LOCATION_ATTRIB6_hashing,STORE_LOCATION_ATTRIB7_hashing,STORE_LOCATION_ATTRIB8_hashing,STORE_LOCATION_ATTRIB9_hashing,STORE_LOCATION_ATTRIB10_hashing,STORE_LOCATION_ATTRIB11_hashing,STORE_LOCATION_ATTRIB12_hashing,STORE_LOCATION_ATTRIB13_hashing,STORE_LOCATION_ATTRIB14_hashing,STORE_LOCATION_ATTRIB15_hashing,STORE_LOCATION_ATTRIB16_hashing,STORE_LOCATION_ATTRIB17_hashing,STORE_LOCATION_ATTRIB18_hashing,STORE_LOCATION_ATTRIB19_hashing,STORE_LOCATION_ATTRIB20_hashing,STORE_LOCATION_ATTRIB21_hashing
count,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329,35329
unique,159,1,1,1,1,2,11,3,1,1,1,4,2,4,2,2,2,1,2,1,1,1,1,1,1
top,2019-03-25,15Mar2018:14:08:08,01Jan5999:00:00:00,181F921CC957DF5CEC4BF2C18E4A371F,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,04E5A06E3993F6C9A4BFDEF278636ED5,F5126A9BDB60B4C8F3B1680AF38B33B9,75300434BE16A57FD386B22D19B289D5,BC0EB006F36BC82F8E165D6D4E95F207,CB5A0648A9D14B22A85E039557A09205,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
freq,240,35329,35329,35329,35329,28869,9226,15213,35329,35329,35329,14249,30710,29461,30710,30710,30710,35329,30713,35329,35329,35329,35329,35329,35329


In [15]:
data.drop(data.columns[data.nunique() == 1], axis=1, inplace=True)

- ### Заполняем пропуски

In [16]:
print((data.demand[(data.PRICE_AFTER_DISC.isna()) & (data.PRICE_REGULAR.isna())] == 0).sum())
print((data.demand==0).sum())

822
5454


In [17]:
data[data.columns[data.isnull().sum() > 0]].describe()

,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
count,34129.000000,35159.000000,34217.000000,34212.000000,35159.000000
mean,12.226552,0.206434,1167.679357,1155.778351,0.907677
std,32.585194,0.433393,1046.828551,1057.912830,0.289486
min,0.000000,0.000000,49.000000,8.647059,0.000000
25%,2.000000,0.000000,284.290000,199.000000,1.000000
50%,6.000000,0.000000,1000.000000,1000.000000,1.000000
75%,12.000000,0.000000,2000.000000,2000.000000,1.000000
max,1160.000000,2.000000,3000.000000,3000.000000,1.000000


Заполним пропуски в `PROMO1_FLAG` и `AUTORIZATION_FLAG` нулями, а в `PRICE_REGULAR` и `PRICE_AFTER_DISC` средним значением по продукту. 


In [18]:
nan_indices = data[(data.demand.isna()) & (data.PRICE_REGULAR.isna())].index
len(nan_indices)

120

In [19]:
data.fillna({'PROMO1_FLAG': 0, 'AUTORIZATION_FLAG': 0}, inplace=True)

In [20]:
data.groupby(['product_rk', 'store_location_rk']).mean()[['PRICE_REGULAR', 'PRICE_AFTER_DISC']]

PRICE_REGULAR  PRICE_AFTER_DISC
product_rk store_location_rk                                 
40369      317                   500.000000        500.000000
           355                   500.000000        500.000000
           380                   500.000000        500.000000
           425                   500.000000        500.000000
           453                   500.000000        500.000000
...                                     ...               ...
96212      1326                   81.074534         61.928174
           1328                   77.661334         59.492329
           1347                   81.267632         60.530233
           1363                   82.497366         61.430322
           1380                   97.081039         65.835838

[240 rows x 2 columns]

In [21]:
data['PRICE_REGULAR'] = data.groupby(['product_rk', 'store_location_rk'])['PRICE_REGULAR'].transform(
    lambda grp: grp.fillna(np.mean(grp))
)
data['PRICE_AFTER_DISC'] = data.groupby(['product_rk', 'store_location_rk'])['PRICE_AFTER_DISC'].transform(
    lambda grp: grp.fillna(np.mean(grp))
)

In [22]:
data.isnull().sum()[data.isnull().sum()>0]

demand    1200
dtype: int64

- ### Делаем Time Series

In [23]:
data['period_start_dt'] = pd.to_datetime(data['period_start_dt'])

In [24]:
data['month'] = data.period_start_dt.dt.month
data['day'] = data.period_start_dt.dt.day

In [25]:
# turn to object
data['product_rk'] = data['product_rk'].astype('object')
data['store_location_rk'] = data['store_location_rk'].astype('object')

In [26]:
# adding time index
data["time_idx"] = data["period_start_dt"].dt.year * 12 + data["period_start_dt"].dt.month
data["time_idx"] -= data["time_idx"].min()

Посмотрим на временные ряды, группируем по магазинам

In [ ]:
for d in data.store_location_rk.unique():
    display(px.line(data.groupby(['store_location_rk'])['product_rk', 'period_start_dt', 'demand'].get_group(d),
                                    color='product_rk', x='period_start_dt', y='demand', title=f"store: {d}"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



Судя по графикам магазины 533, 540, 1005, 1080, 1202 закрылись, либо перестали продавать данные товары. Поэтому занулим им спросы в финальном сабмите

In [28]:
def closed(x):
  if x in [533, 540, 1005, 1080, 1202]:
    return x

In [29]:
forecasts = data['demand'][data[data.store_location_rk.apply(closed).isna() == False].index]
zero_forecasts = pd.DataFrame(forecasts[forecasts.isna()])
zero_forecasts['demand'] = 0

# 3. Предсказываем

In [30]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

In [32]:
data.head()

,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,STORE_LOCATION_LVL_RK4,STORE_LOCATION_ATTRIB2_hashing,STORE_LOCATION_ATTRIB3_hashing,STORE_LOCATION_ATTRIB4_hashing,STORE_LOCATION_ATTRIB8_hashing,STORE_LOCATION_ATTRIB9_hashing,STORE_LOCATION_ATTRIB10_hashing,STORE_LOCATION_ATTRIB11_hashing,STORE_LOCATION_ATTRIB12_hashing,STORE_LOCATION_ATTRIB13_hashing,STORE_LOCATION_ATTRIB15_hashing,month,day,time_idx
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,
15,40369,317,2016-12-19,50.0,0.0,500.000000,500.000000,0.0,189,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,04E5A06E3993F6C9A4BFDEF278636ED5,19517FD4F1E15806B405B9D730EEADCE,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,FC5FD090934CE685C1C12E5E0311F1D4,12,19,0
16,40370,317,2016-12-19,44.0,0.0,1000.000000,1000.000000,0.0,189,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,04E5A06E3993F6C9A4BFDEF278636ED5,19517FD4F1E15806B405B9D730EEADCE,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,FC5FD090934CE685C1C12E5E0311F1D4,12,19,0
17,40372,317,2016-12-19,13.0,0.0,2000.000000,2000.000000,0.0,189,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,04E5A06E3993F6C9A4BFDEF278636ED5,19517FD4F1E15806B405B9D730EEADCE,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,FC5FD090934CE685C1C12E5E0311F1D4,12,19,0
18,40373,317,2016-12-19,6.0,0.0,3000.000000,3000.000000,0.0,189,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,04E5A06E3993F6C9A4BFDEF278636ED5,19517FD4F1E15806B405B9D730EEADCE,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,FC5FD090934CE685C1C12E5E0311F1D4,12,19,0
19,46272,317,2016-12-19,34.0,0.0,235.425832,176.701533,0.0,189,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,04E5A06E3993F6C9A4BFDEF278636ED5,19517FD4F1E15806B405B9D730EEADCE,B2581C791DBB8894BFDCFB66B60A17ED,2C3957E2A91B1D2BEE334248B387E402,67B1193B31375E3C959BB5246989D0CA,67B1193B31375E3C959BB5246989D0CA,17AC9AE74D16FBC665FFB4CFC2DF24F5,FC5FD090934CE685C1C12E5E0311F1D4,12,19,0


- ### Stacking Regressor - оказался лучшим

In [33]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('hgb', HistGradientBoostingRegressor(loss='least_absolute_deviation', learning_rate=0.17, max_depth=15,
                                        l2_regularization=1.5, max_iter=100000, random_state=13)))
	level0.append(('cart', RandomForestRegressor(n_estimators=100, random_state=1)))
	level0.append(('svm', SVR()))
	# define meta learner model
	level1 = LinearRegression()
	# define the stacking ensemble
	model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
	return model

In [34]:
# создадим дамми и выбросим дату, чтоб не мешала 
dum = pd.get_dummies(data, drop_first=True)
dum.drop('period_start_dt', axis=1, inplace=True)

In [36]:
%%time

train = dum[dum.demand.isna() == False]
test_X = dum[dum.demand.isna()].drop('demand', axis=1).to_numpy()
X = train.drop('demand', axis=1).to_numpy()
y = train.demand.to_numpy()


model = get_stacking()
model.fit(X, y)
pred = model.predict(test_X)

predicted = pd.DataFrame(pred, index=dum[dum.demand.isna()].index)

CPU times: user 7h 54min 55s, sys: 9min 39s, total: 8h 4min 34s
Wall time: 4h 14min 59s


In [37]:
predicted.loc[zero_forecasts.index]

,0
Unnamed: 0,
8337,0.667713
8338,0.179596
8339,0.684786
8340,0.916636
8341,0.907075
...,...
29704,0.230543
29705,0.758619
29706,1.024386


*Как видно предикты в закрытых магазинах чуть больше нуля, что в целом хорошо, но для большей точности заменим их на 0*

In [38]:
predicted.loc[zero_forecasts.index] = 0

 ## Записываем сабмишн

In [39]:
sample = pd.read_csv('/content/hw2_data_analysis/demand-forecasting-in-retail/sample_submission.csv')
sample.head()

,id,predicted
0,908,0
1,909,0
2,910,0
3,911,0
4,912,0


Запишем в sample и сохраним файл

In [40]:
sample['predicted'] = predicted[0].to_numpy()
sample.to_csv('stacking_regressor_all.csv', index=False)

**Итоговая проверка:** посмотрим на графиках, насколько логичными получились предсказания

In [41]:
data_predicted = data.copy()
data_predicted['demand'][data_predicted.demand.isna()] = sample.predicted.to_numpy()

for d in data_predicted.store_location_rk.unique():
    display(px.line(data_predicted.groupby(['store_location_rk'])['product_rk', 'period_start_dt', 'demand'].get_group(d),
                                    color='product_rk', x='period_start_dt', y='demand', title=f"store: {d}"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

